In [48]:
from gc import callbacks
import os
from skimage.measure import label, regionprops_table, regionprops
from PIL import Image
from skimage.io import imread
from skimage.transform import rescale, resize
# for visualizations
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np # for using np arrays
import glob
# for bulding and running deep learning model
from stardist.models import StarDist2D 
from csbdeep.utils import normalize
from skimage.segmentation import find_boundaries

In [49]:
def add_data (df_old, data_new):
    df_new_row = pd.DataFrame(data=data_new, columns=['Row','Column','Field','Skan','Count'])
    df1 = pd.concat([df_old,df_new_row], ignore_index=True)
    return df1
def add_row (df_old, row):
    df1 = pd.concat([df_old,row], ignore_index=True)
    return df1

In [50]:
def LoadData(path1):
    """
    Looks for relevant filenames in the shared path
    Returns 2 lists for original and masked files respectively
    """
    # Make a list for images and masks filenames
    file_data = open("CellsCount.csv","w+")
    file_data.write('Row,Column,Field,Skan,Count\n')
    bw_img = []
    df_new = pd.DataFrame(columns=['Row','Column','Field','Skan','Count'])
    r = re.compile(r'r(\d+)c(\d+)f(\d+)p(\d+)-ch(\d+)sk(\d+)')
    def custom_key(x):
        val = int(r.search(x).group(5)) * 100_000_000 + int(r.search(x).group(1)) * 10_000_000 + int(r.search(x).group(2)) * 100_000 + int(r.search(x).group(3)) * 1_000 + int(r.search(x).group(6))
        return val
    def custom_key2(x):
        val = int(r.search(x).group(1)) * 1_000_000 + int(r.search(x).group(2)) * 100_000 + int(r.search(x).group(3)) * 1_000 + int(r.search(x).group(6))
        return val
    def custom_key3(x):
        val =  int(r.search(x).group(6))
        return val
    
    # Read the images folder like a list
    for filename in sorted(glob.glob(path1 + '*.tiff'), key = lambda x: custom_key(x)):
        #print (int(r.search(filename).group(6)))
        
        bw = imread(filename)
        unique, counts = np.unique(bw, return_counts=True)
        dd = dict(zip(unique, counts))
        count = int(dd[255]/40)
        #row_n_add = [int(r.search(filename).group(1)), int(r.search(filename).group(2)),int(r.search(filename).group(3)), int(r.search(filename).group(6)), count]
        #df_new_row = pd.DataFrame(data=row_n_add, columns = ['Row','Column','Field','Skan','Count'])
        file_data.write(r.search(filename).group(1) + ',' + r.search(filename).group(2) + ',' + r.search(filename).group(3) + ',' + r.search(filename).group(6) + ',' + str(count)+ '\n')
        #df_new = add_row(df_new, df_new_row)
        
        
    # Sort the lists to get both of them in same order (the dataset has exactly the same name for images and corresponding masks)
    #bw_img.sort(key = lambda x: custom_key2(x))
    
    #analyse_list = []
    #for filename,i in enumerate(bw_img):
    
    return df_new



In [51]:
def PreprocessData(bright, fluor, target_shape_img, target_shape_mask, path1, path2):
    """
    Processes the images and mask present in the shared list and path
    Returns a NumPy dataset with images as 3-D arrays of desired size
    Please note the masks in this dataset have only one channel
    
    
     image_bright_field  = imread(filename)
            image_bright_field = np.float64(image_bright_field)
            image_bright_field = image_bright_field * 255.0 / image_bright_field.max()
            image_bright_field = np.uint8(image_bright_field)
            
            image_fluor  = imread(filename)
            image_fluor = np.float64(image_fluor)
            image_fluor = image_fluor * 255.0 / image_fluor.max()
            image_fluor = np.uint8(image_fluor)
    """
    # Pull the relevant dimensions for image and mask
    m = len(bright)                    # number of images
    i_h, i_w, i_c = target_shape_img   # pull height, width, and channels of image
    m_h, m_w, m_c = target_shape_mask  # pull height, width, and channels of mask
    print (i_h, i_w, i_c)
    print (m_h, m_w, m_c)
    # Define X and Y as number of images along with shape of one image
    X = np.zeros((m, i_h, i_w, i_c), dtype=np.float32)
    y = np.zeros((m, m_h, m_w, m_c), dtype=np.int32)
    model = StarDist2D.from_pretrained('2D_versatile_fluo')
    # Resize images and masks
    for file in bright:
        # convert image into an array of desired shape (3 channels)
        index = bright.index(file)
        #path = os.path.join(path1, file)
        
        #image_bright_field = Image.open(file)#.convert('RGB') #imread(file) #Image.open(file).convert('RGB')
        image_bright_field = imread(file)
        image_bright_field = resize(image_bright_field, (i_h, i_w),
                       anti_aliasing=True)
        #print (image_bright_field.shape)
        #image_bright_field = image_bright_field.resize((i_h, i_w))
        image_bright_field = np.reshape(image_bright_field,(i_h, i_w, i_c)) 
        #image_bright_field = single_img/256.
        image_bright_field = np.float64(image_bright_field)
        image_bright_field = image_bright_field / image_bright_field.max()
        #image_bright_field = np.uint8(image_bright_field)
        X[index] = image_bright_field
        #print (image_bright_field.shape)
        
        # convert mask into an array of desired shape (1 channel)
        single_mask_ind = fluor[index]
        #image_fluor  = Image.open(file) # imread(single_mask_ind)
        image_fluor  = imread(single_mask_ind)
        
        ###new
        labels, _ = model.predict_instances(normalize(image_fluor))
        img_cont = find_boundaries(labels)
        labels = np.where(img_cont != 0, 0, labels)
        labels = np.where(labels != 0, 1, labels)
        labels = resize(labels, (i_h, i_w),
                       anti_aliasing=True)
        #image_fluor = image_fluor.resize((i_h, i_w))
        labels = np.reshape(labels,(i_h, i_w, i_c)) 
        labels = np.where(labels != 0, 1, labels)
        labels = np.uint8(labels)
        #image_bright_field = single_img/256.
        y[index] = labels
        # image_fluor = resize(image_fluor, (i_h, i_w),
        #                anti_aliasing=True)
        # #image_fluor = image_fluor.resize((i_h, i_w))
        # image_fluor = np.reshape(image_fluor,(i_h, i_w, i_c)) 
        # image_fluor = np.float64(image_fluor)
        # image_fluor = image_fluor * 31.0 / image_fluor.max()
        # image_fluor = np.uint8(image_fluor)
        # #image_bright_field = single_img/256.
        # y[index] = image_fluor
        
    return X, y

In [52]:
LoadData('/Volumes/nikitas_usb/Model CNN/pred/')

,Row,Column,Field,Skan,Count


In [ ]:
data = pd.read_csv("CellsCount.csv", sep = ',')

In [ ]:
rows = data['Row'].unique()
columns = data['Column'].unique()
for row in rows:
    row_cur = data[data['Row'] == row]
